# Server

## Setting variables

In [1]:
rounds = 100
num_users = 10 # number of clients
target_test_acc = 0.99
lrs = [0.1]

C = 1
E = 5
B = 10 # 'all' for a single minibatch

In [2]:
import os

import socket
import struct
import pickle
import sys

from threading import Thread
from threading import Lock

import copy

import logging
import math
import random
import re
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from datetime import timedelta
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tqdm import tqdm
tf.get_logger().setLevel(logging.ERROR)

2023-05-09 03:44:51.216984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 03:44:52.156494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Device

In [3]:
# if tf.test.gpu_device_name():
#     device = "gpu"
# else:
#     device = "cpu"

# print(device)

## Model

A CNN with two 5x5 convolution layers (the first with 32 channels, the second with 64, each followed with 2x2 max pooling), a fully connected layer with 512 units and ReLu activation, and a final softmax output layer (1,663,370 total parameters)

In [4]:
class CNN:
    @staticmethod
    def build(input_shape):
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(filters=64, padding='same', kernel_size=(5,5), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(10, activation='softmax'))
        return model

initialize global model

In [5]:
model = CNN()
  
global_model = model.build((28,28,1))
initial_weights = global_model.get_weights()

global_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 512)               1

2023-05-09 03:44:53.972154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [6]:
# # Save the entire model as a SavedModel.
# checkpoint_path = "training/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

global_model.save('saved_model/global_model')

# Load data

In [7]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [8]:
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
train_batched = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(len(y_train)) # for testing on train set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

## variables

In [10]:
global global_weights    

clientsoclist = [0]*num_users

start_time = 0
weight_count = 0

global_weights = initial_weights
weights_list = {}

lock = Lock()

## Socket initialization
### Set host address and port number

### Required socket functions

In [11]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    l_send = len(msg)
    msg = struct.pack('>I', l_send) + msg
    sock.sendall(msg)
    return l_send

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [12]:
def average_weights(weights_list):
    avg_weights = list()
    for j in range(len(global_weights)):
        weights = [weights_list[k][j] for k in range(num_users)]
        layer_mean = tf.math.reduce_mean(weights, axis=0)
        avg_weights.append(layer_mean)
        
    return avg_weights

## Receive users for aggregation

In [13]:
def receive(userid, r, num_users, conn): #thread for receive clients
    global weight_count
    global global_weights
    global weights_list
    
    msg = {
        'current_round': r,
        'rounds': rounds,
        'client_id': userid,
        'weight': global_weights
    }
    distribute = send_msg(conn, msg)    #send global weight
    receive = recv_msg(conn)    # get weights from clients
    with lock:
        weights_list[userid] = receive['weight']
        if receive['current_round'] == r:
            weight_count += 1
        
        if weight_count == num_users:
            global_weights = average_weights(weights_list)
            weight_count = 0

## Thread define

In [14]:
def run_thread(func, num_user, r):
    global clientsoclist
    global start_time
    
    thrs = []
    print("timer start!")
    start_time = time.time()    # store start time
    for i in range(num_user):
        conn, addr = s.accept()
        print('Conntected with', addr)
        # append client socket on list
        clientsoclist[i] = conn
        args = (i, r, num_user, conn)
        thread = Thread(target=func, args=args)
        thrs.append(thread)
        thread.start()
    for thread in thrs:
        thread.join()
    end_time = time.time()  # store end time
    print("TrainingTime: {} sec".format(end_time - start_time))

In [15]:
host_name = socket.gethostbyname(socket.gethostname())
# host_name = '172.31.2.147'
port_number = 12345
print(host_name)

172.31.26.96


In [16]:
print(host_name)

172.31.26.96


### Open the server socket

In [17]:
s = socket.socket()
s.bind((host_name, port_number))
s.listen(10)

## Comunication overhead

In [ ]:
loss='categorical_crossentropy'
metrics = ['accuracy']
cce = tf.keras.losses.CategoricalCrossentropy()

result_per_lr = {}
start = time.time()

dir = ''
    
for lr in lrs:
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    
    print('\nlearning rate: {}'.format(lr))
    
    for r in range(rounds):
        train_loss = 0
        train_acc = 0
        test_loss = 0
        test_acc = 0
        
        run_thread(receive, num_users, r)
    
        # evaluate current round
        start = time.time()
        global_model.set_weights(global_weights)
        
        # test global model on full training set
        for (X,y) in train_batched:
            preds = global_model.predict(X)
            train_loss = cce(y, preds)
            train_acc = accuracy_score(tf.argmax(preds, axis=1), tf.argmax(y, axis=1))
            train_losses.append(train_loss.numpy())
            train_accs.append(train_acc)

        # test global model on testing set
        for(X, y) in test_batched:
            preds = global_model.predict(X)
            test_loss = cce(y, preds)
            test_acc = accuracy_score(tf.argmax(preds, axis=1), tf.argmax(y, axis=1))
            test_losses.append(test_loss.numpy())
            test_accs.append(test_acc)

        elapsed = (time.time() - start)

        print('comm_round: {}/{} | test_acc: {:.3%} | test_loss: {:.3} | train_acc: {:.3%} | train_loss: {:.3} | elapsed: {}'.format(r+1, rounds, test_acc, test_loss, train_acc, train_loss, timedelta(seconds=elapsed)))
        print('\n')
        global_model.save('saved_model/global_model')
        
    result_per_lr[lr] = {
        'train_accs' : train_accs,
        'test_accs' : test_accs,
        'train_losses' : train_losses,
        'test_losses' : test_losses
                          }
    
    


learning rate: 0.1
timer start!
Conntected with ('172.31.26.96', 59218)
Conntected with ('172.31.26.96', 59232)
Conntected with ('172.31.26.96', 59246)
Conntected with ('172.31.26.38', 39090)
Conntected with ('172.31.26.38', 39100)
Conntected with ('172.31.26.38', 39110)
Conntected with ('172.31.23.109', 40580)
Conntected with ('172.31.23.109', 36106)
Conntected with ('172.31.23.109', 36122)
Conntected with ('172.31.24.39', 42912)
TrainingTime: 166.5627269744873 sec


2023-05-09 03:47:42.606024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [60000,10]
	 [[{{node Placeholder/_1}}]]


  1/313 [..............................] - ETA: 24s

2023-05-09 03:47:58.476050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [10000,10]
	 [[{{node Placeholder/_1}}]]


313/313 [==============================] - 3s 8ms/step
comm_round: 1/100 | test_acc: 53.910% | test_loss: 1.86 | train_acc: 53.253% | train_loss: 1.87 | elapsed: 0:00:18.861891




...

timer start!
Conntected with ('172.31.24.39', 58510)
Conntected with ('172.31.26.38', 36100)
Conntected with ('172.31.23.109', 59718)
Conntected with ('172.31.26.38', 36108)
Conntected with ('172.31.23.109', 59724)
Conntected with ('172.31.26.38', 36114)
Conntected with ('172.31.23.109', 59736)
Conntected with ('172.31.26.96', 44768)
Conntected with ('172.31.26.96', 50822)
Conntected with ('172.31.26.96', 42848)
TrainingTime: 292.00776743888855 sec
313/313 [==============================] - 10s 31ms/step
comm_round: 34/100 | test_acc: 98.460% | test_loss: 0.0478 | train_acc: 98.632% | train_loss: 0.0407 | elapsed: 0:01:01.866038




timer start!
Conntected with ('172.31.24.39', 46900)
Conntected with ('172.31.26.38', 36948)
Conntected with ('172.31.23.109', 60656)
Conntected with ('172.31.23.109', 60662)
Conntected with ('172.31.26.38', 36956)
Conntected with ('172.31.23.109', 60666)
Conntected with ('172.31.26.38', 36962)
Conntected with ('172.31.26.96', 51794)
Conntected with ('172.31.26.96', 51804)
Conntected with ('172.31.26.96', 48376)
TrainingTime: 270.45665884017944 sec
313/313 [==============================] - 10s 31ms/step
comm_round: 35/100 | test_acc: 98.570% | test_loss: 0.0449 | train_acc: 98.715% | train_loss: 0.037 | elapsed: 0:01:10.383458




timer start!
Conntected with ('172.31.24.39', 49090)
Conntected with ('172.31.26.38', 53340)
Conntected with ('172.31.23.109', 37648)
Conntected with ('172.31.26.38', 53878)
Conntected with ('172.31.23.109', 37650)
Conntected with ('172.31.23.109', 37654)
Conntected with ('172.31.26.38', 53890)
Conntected with ('172.31.26.96', 35094)
Conntected with ('172.31.26.96', 35108)
Conntected with ('172.31.26.96', 49078)
TrainingTime: 277.59100246429443 sec
313/313 [==============================] - 10s 32ms/step
comm_round: 36/100 | test_acc: 98.510% | test_loss: 0.0454 | train_acc: 98.730% | train_loss: 0.0375 | elapsed: 0:01:10.564137




timer start!
Conntected with ('172.31.24.39', 33714)
Conntected with ('172.31.26.38', 36504)
Conntected with ('172.31.23.109', 36616)
Conntected with ('172.31.26.38', 36520)
Conntected with ('172.31.23.109', 36632)
Conntected with ('172.31.23.109', 36646)
Conntected with ('172.31.26.38', 36528)
Conntected with ('172.31.26.96', 46752)
Conntected with ('172.31.26.96', 46768)
Conntected with ('172.31.26.96', 46776)
TrainingTime: 300.65967559814453 sec
313/313 [==============================] - 10s 32ms/step
comm_round: 37/100 | test_acc: 98.500% | test_loss: 0.0476 | train_acc: 98.612% | train_loss: 0.0409 | elapsed: 0:01:10.653892




timer start!
Conntected with ('172.31.24.39', 34852)
Conntected with ('172.31.26.38', 40034)
Conntected with ('172.31.23.109', 53234)
Conntected with ('172.31.26.38', 40572)
Conntected with ('172.31.23.109', 53236)
Conntected with ('172.31.23.109', 53250)
Conntected with ('172.31.26.38', 40588)
Conntected with ('172.31.26.96', 58772)
Conntected with ('172.31.26.96', 35246)
Conntected with ('172.31.26.96', 58516)


In [ ]:
with open(dir+'result_per_lr_{}_{}_{}_{}.pickle'.format(B,C,E, lr), 'wb') as handle:
    pickle.dump(result_per_lr, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def get_plotted_metrics(result_per_lr, c_rounds):
    plotted_train_accs= []
    plotted_test_accs = []
    plotted_train_losses = []
    plotted_test_losses = []

    for c in range(c_rounds):
        best_train_acc = 0
        best_test_acc = 0
        best_train_loss = math.inf
        best_test_loss = math.inf
        for lr in result_per_lr.keys():
            best_train_acc = max(best_train_acc, result_per_lr[lr]['train_accs'][c])
            best_test_acc = max(best_test_acc, result_per_lr[lr]['test_accs'][c])
            best_train_loss = min(best_train_loss, result_per_lr[lr]['train_losses'][c])
            best_test_loss = min(best_test_loss, result_per_lr[lr]['test_losses'][c])

    if c == 0:
        plotted_train_accs.append(best_train_acc)
        plotted_test_accs.append(best_test_acc)
        plotted_train_losses.append(best_train_loss)
        plotted_test_losses.append(best_test_loss)
    else:
        if plotted_train_accs[-1] > best_train_acc:
            plotted_train_accs.append(plotted_train_accs[-1])
        else:
            plotted_train_accs.append(best_train_acc)

        if plotted_test_accs[-1] > best_test_acc:
            plotted_test_accs.append(plotted_test_accs[-1])
        else:
            plotted_test_accs.append(best_test_acc)

        if plotted_train_losses[-1] < best_train_loss:
            plotted_train_losses.append(plotted_train_losses[-1])
        else:
            plotted_train_losses.append(best_train_loss)

        if plotted_test_losses[-1] < best_test_loss:
            plotted_test_losses.append(plotted_test_losses[-1])
        else:
            plotted_test_losses.append(best_test_loss)
    
    return plotted_train_accs, plotted_test_accs, plotted_train_losses, plotted_test_losses

In [ ]:
plotted_train_accs, plotted_test_accs, plotted_train_losses, plotted_test_losses = get_plotted_metrics(result_per_lr, c_rounds)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(1, len(plotted_train_accs)+1), plotted_train_accs, label='train')
ax.plot(range(1,len(plotted_test_accs)+1), plotted_test_accs, label='test')
ax.set_xticks(np.arange(0, len(plotted_test_accs)+1, 100))
ax.axhline(y=target_test_acc, color='grey', linestyle='-', linewidth=0.5)
ax.set_ylabel('accuracy')

ax.set_xlabel('communication rounds')
ax.set_title('B={}, C={}, E={}, Model={}, Data={}'.format(B, C, E, MODEL, DATA))
ax.legend()

ax2 = ax.twinx()
ax2.set_ylim(ax.get_ylim())
ax2.set_yticks([target_test_acc])

plt.show()

In [ ]:
n_round_at_target = None
for i,acc in enumerate(plotted_test_accs):
    if acc>=target_test_acc:
        print("the number of rounds to achieve target test-accuracy: ")
        n_round_at_target = i+1
        print(n_round_at_target)
        break;

In [ ]:
loss_at_target = None
for i,loss in enumerate(plotted_test_losses):
    if n_round_at_target and i==n_round_at_target-1:
        print("loss at target test-accuracy: ")
        loss_at_target = loss
        print(loss_at_target)
        break;

In [ ]:
# end_time = time.time()  # store end time
# print("TrainingTime: {} sec".format(end_time - start_time))

In [ ]:
end_time = time.time()  # store end time
print("WorkingTime: {} sec".format(end_time - start_time))